In [1]:
import os
import json
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from math import isclose, ceil, floor
from statistics import mean
from tqdm.autonotebook import tqdm
plt.rcParams['figure.dpi'] = 150

/tmp/ipykernel_33482/2035716161.py:10: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
%matplotlib qt

In [3]:
search_path = '../data/'
_, dirs, _ = next(os.walk(search_path))

In [4]:
dirs = sorted(dirs)
dirs

['2022-05-16-10-31-07',
 '2022-05-16-11-02-59',
 '2022-09-05-16-29-36',
 '2022-09-06-12-25-01',
 '2022-09-06-12-55-08',
 '2022-09-06-17-33-51',
 '2022-09-06-18-42-00',
 '2022-09-06-20-30-16',
 '2022-09-16-15-21-02',
 '2022-09-16-16-05-43']

In [5]:
sensors = ['rear_left'] # 'front_right','front_left','rear_right','rear_left'

In [6]:
pbar = tqdm(total=len(dirs))
for d in dirs:
    ref = pd.read_csv(search_path + d + '/inspva.csv')
    ref['speed'] = np.sqrt(np.square(ref.loc[:,'east_velocity'].to_numpy()) +
                           np.square(ref.loc[:,'north_velocity'].to_numpy()) +
                           np.square(ref.loc[:,'up_velocity'].to_numpy()))
    ref.loc[:,'Time'] = ref.loc[:,'header.stamp.secs'] + ref.loc[:,'header.stamp.nsecs'] * 1e-9
    ref_time = ref['Time'].to_numpy() 
    
    for sensor in sensors:
        data = []
        s = pd.read_csv(search_path + d + '/' + sensor + '.csv')
        dummy = [[20.,20.,0.,0.,0.,0.]]
        
        # Syncing with reference time
        grouped = s.groupby(['Time'], sort=True)    
        for t in grouped.groups.keys():
            group = grouped.get_group(t)
            temp = np.abs(ref_time - t)
            id = np.argmin(temp)
            if temp[id] < 0.02:
                if group.shape[0] >= 3:
                    data.append([t, group.loc[:,['x','y','z','rcs','snr','speed']].values.tolist(), ref['speed'].iat[id]])
                # else:
                #     data.append([t, dummy, 1e-3])
        
        # Finding gaps in the data
        timestamps = []
        hz = []
        for i in range(1,len(data)):
            curr_t, _, _ = data[i]
            prev_t, _, _ = data[i-1]
            dt = curr_t - prev_t
            if isclose(dt, 0.05, abs_tol=0.01):
                hz.append(dt)
            timestamps.append(prev_t)
            if i == len(data)-1:
                timestamps.append(curr_t)
                
        # Creating continuous timeline
        init_t = timestamps[0]
        end_t  = timestamps[-1]
        step = mean(hz)
        timeline = np.arange(init_t,end_t+step,step)
        
        # Indexes for augmenting dataset
        valid = np.zeros((timeline.shape[0],3))
        valid[:,0] = timeline
        for i, t in enumerate(timestamps):
            temp = np.abs(timeline - t)
            id = np.argmin(temp)
            if isclose(timeline[id],t,abs_tol=0.01):
                valid[id,1:] = [1,i]
                
        # Augmenting the dataset
        augmented = []
        for i in range(valid.shape[0]):
            t,status,idx = valid[i,:]
            status = int(status)
            idx = int(idx)
            if status:
                _, curr_radar, curr_ref = data[idx]
                augmented.append([t,curr_radar,curr_ref,status])
            else:
                augmented.append([t, dummy, 1e-3, status])
        
        # Saving text file
        with open('../synched/rl_v1/' + d + '-' + sensor + '.txt', 'w') as fp:
            augmented = sorted(augmented, key=lambda x: x[0])
            for dt in augmented:
                fp.write(str(dt)+'\n')
    
    pbar.update(n=1)

100%|███████████████████████████████████████████| 10/10 [03:03<00:00, 20.89s/it]